In [1]:
from importlib import reload
from pathlib import Path

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import plotutils as pu
reload(pu)

%matplotlib widget

In [2]:
def read(name):
    jobs = pd.read_csv(Path('../log/bs20') / name / 'jobs.csv')
    return jobs

def write(name, df):
    df.to_csv(Path('../log/bs20') / name / 'jobs.csv', index=False)
    pass

def compare(j1, j2, n1, n2, bs, figsize=(8, 8)):
    fig = plt.figure(figsize=figsize, constrained_layout=True)
    axs = fig.subplot_mosaic('''AD
                                AD
                                BE
                                BE
                                CF
                                ''')
    ax = pu.cdf(j1.Length, ax=axs['A'])
    ax.set_ylabel('CDF')
    ax.set_xlabel('Length ('+n1+')')
    
    ax = pu.cdf(j1.Latency, ax=axs['B'])
    ax.set_ylabel('CDF')
    ax.set_xlabel('Latency ('+n1+')')
    
    # percentage
    ax = axs['C']
    total = len(j1)
    states = ['past_due', 'done']
    pos = np.arange(len(states))
    data = [
        len(j1.State[j1.State == state]) / total
        for state in states
    ]
    ax.barh(pos, data, align='center')
    for x, y in zip(data, pos):
        pu.bar_show_data(x, y, data_y=x, fmt='{:.0%}', ax=ax, xytext=(4, 0), horizontalalignment='left', verticalalignment='center')
    pu.cleanup_axis_categorical(ax.yaxis, states)
    pu.cleanup_axis_percent(ax.xaxis, xmax=1.0)
    ax.set_xlabel('Percentage ('+n1+')')
    
    ax = pu.cdf(j2.Length, ax=axs['D'])
    ax.set_ylabel('CDF')
    ax.set_xlabel('Length ('+n2+')')
    
    ax = pu.cdf(j2.Latency, ax=axs['E'])
    ax.set_ylabel('CDF')
    ax.set_xlabel('Latency ('+n2+')')
    
    # percentage
    ax = axs['F']
    total = len(j2)
    states = ['past_due', 'done']
    pos = np.arange(len(states))
    data = [
        len(j2.State[j2.State == state]) / total
        for state in states
    ]
    ax.barh(pos, data, align='center')
    for x, y in zip(data, pos):
        pu.bar_show_data(x, y, data_y=x, fmt='{:.0%}', ax=ax, xytext=(4, 0), horizontalalignment='left', verticalalignment='center')
    pu.cleanup_axis_categorical(ax.yaxis, states)
    pu.cleanup_axis_percent(ax.xaxis, xmax=1.0)
    ax.set_xlabel('Percentage ('+n2+')')
    # utilizations
    #ax = axs['U']
    size = len(j2)
    deadline = int(j2.iloc[0]['Deadline'] - j2.iloc[0]['Admitted'])
    print(deadline)
    fig.suptitle("total("+str(size)+")_bs("+str(bs)+")_ddl("+str(deadline)+")")
    return fig, axs

In [3]:
fifo = read('fifo')
fifo['Latency'] = fifo.Finished - fifo.Admitted
# fifo.loc[fifo["Finished"]>fifo["Deadline"], "State"]= "past_due"
# fifo.loc[fifo["Finished"]>fifo["Deadline"], "Latency"]= np.nan
write('fifo', fifo)

In [4]:
!python async_dynamic_test.py ../log/bs20/fifo/jobs.csv --output ../log/bs20/clipper/latency.csv

21-08-02:22:16:53 INFO     [docker_container_manager.py:192] [default-cluster] Starting managed Redis instance in Docker
21-08-02:22:17:06 INFO     [docker_container_manager.py:294] [default-cluster] Metric Configuration Saved at /private/var/folders/j5/b2f72t413y7bry9wv3plqml00000gn/T/tmpgeddm4vw.yml
21-08-02:22:17:10 INFO     [clipper_admin.py:162] [default-cluster] Clipper is running
21-08-02:22:17:10 INFO     [clipper_admin.py:236] [default-cluster] Application fake-model was successfully registered
21-08-02:22:17:10 INFO     [deployer_utils.py:41] Saving function to /var/folders/j5/b2f72t413y7bry9wv3plqml00000gn/T/tmp4e73nj36clipper
21-08-02:22:17:10 INFO     [deployer_utils.py:51] Serialized and supplied predict function
21-08-02:22:17:10 INFO     [python.py:192] Python closure saved
21-08-02:22:17:10 INFO     [python.py:210] Using Python 3.7 base image
21-08-02:22:17:10 INFO     [clipper_admin.py:534] [default-cluster] Building model Docker image with model data from /var/folder

In [5]:
clipper = fifo.copy()
clipper_rst = pd.read_csv(Path('../log/bs20/clipper/latency.csv'))
clipper["Started"] = clipper_rst["Timestamp"]
clipper["Latency"] = clipper_rst["LatencyUS"]/1000
clipper["Finished"] = clipper["Started"] + clipper["Latency"]
clipper["State"] = clipper_rst["State"]
clipper.loc[clipper["Finished"]>clipper["Deadline"], "State"]= "past_due"
clipper.loc[clipper["State"]=="error", "State"]= "past_due"
# clipper.loc[clipper["Finished"]>clipper["Deadline"], "Latency"]= np.nan
write('clipper', clipper)

In [6]:
bs = 20
fig, _ = compare(fifo, clipper, "fifo", "clipper", bs, (10,10))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

130
